In [12]:
import argparse
import math
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM, LinfDeepFoolAttack
from advertorch.attacks import LinfSPSAAttack
from src.trainers import Trainer, FGSMTrainer
from robustbench.model_zoo.models import Carmon2019UnlabeledNet
from src.utils import adversarial_accuracy, fgsm_
from src.gradient_masking_tests import gradient_norm, fgsm_pgd_cos_dif, multi_scale_fgsm, gradient_information
import eagerpy as ep
from src.Nets import CIFAR_Wide_Res_Net, CIFAR_Res_Net, CIFAR_Net, ResNet18GradReg
%load_ext autoreload
%autoreload 2
%aimport src.Nets, src.trainers, src.gradient_masking_tests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# setup
device = torch.device("cuda")
batch_size = 128
# remove the normalize
transform = transform = transforms.Compose(
            [transforms.ToTensor()]
)
        
normalized_min = (0 - 0.5) / 0.5
normalized_max = (1 - 0.5) / 0.5
train_dataset = datasets.CIFAR10(root='data', train=True,
                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=True, num_workers=2)
test_dataset = datasets.CIFAR10(root='data', train=False,
                               download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)
classes = classes = ('plane', 'car', 'bird', 'cat',
   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# # Regular CIFAR-10  ResNet Model
model = CIFAR_Res_Net(device).eval()
model.load_state_dict(torch.load("models/cifar_res_net.model"))
step_ll_model = CIFAR_Res_Net(device).eval()
step_ll_model.load_state_dict(torch.load("models/cifar_res_net_20e_stepll.model"))
fgsm_model = CIFAR_Res_Net(device).eval()
fgsm_model.load_state_dict(torch.load("models/cifar_res_net_fgsm06.model"))
# # # CIFAR-10  ResNet Model trained with small FGSM steps
fgsm_model_small = CIFAR_Res_Net(device).eval()
fgsm_model_small.load_state_dict(torch.load("models/cifar_res_net_20e_fgsm03.model"))
gradient_norm_model = CIFAR_Res_Net(device).eval()
gradient_norm_model.load_state_dict(torch.load("models/cifar_res_net_grad_reg_20e_ld2.model"))
# # CIFAR-10  ResNet Model trained through input gradient regularization ld5
gradient_norm_model_strong = CIFAR_Res_Net(device).eval()
gradient_norm_model_strong.load_state_dict(torch.load("models/cifar_res_net_grad_reg_20e_ld5.model"))

<All keys matched successfully>

## Error with perturbation where each coordinate is either -eps or eps

In [17]:
for cur_model, name in zip([model, step_ll_model, fgsm_model, fgsm_model_small, gradient_norm_model], ['normal', 'step ll', 'fgsm', 'fgsm small', 'grad norm']):
    print(name + ': ')
    linearization_error(cur_model, train_dataset, epsilons=[0.003, 0.03, 0.06, 0.1], device=device)

Epsilon 0.003: 0.06999900192022324 error
Epsilon 0.03: 0.7696435451507568 error
Epsilon 0.06: 2.1171889305114746 error
Epsilon 0.1: 6.860876083374023 error
Epsilon 0.003: 0.04398001357913017 error
Epsilon 0.03: 0.36212143301963806 error
Epsilon 0.06: 0.7624298334121704 error
Epsilon 0.1: 1.3676996231079102 error
Epsilon 0.003: 0.24454203248023987 error
Epsilon 0.03: 2.3595657348632812 error
Epsilon 0.06: 5.2559075355529785 error
Epsilon 0.1: 0.7985435128211975 error
Epsilon 0.003: 0.010167415253818035 error
Epsilon 0.03: 0.1098482683300972 error
Epsilon 0.06: 0.20659400522708893 error
Epsilon 0.1: 0.4114561080932617 error
Epsilon 0.003: 0.07267075777053833 error
Epsilon 0.03: 0.8342500925064087 error
Epsilon 0.06: 2.1449363231658936 error
Epsilon 0.1: 8.656185150146484 error
Epsilon 0.003: 0.06968510150909424 error
Epsilon 0.03: 0.7737661004066467 error
Epsilon 0.06: 2.3319265842437744 error
Epsilon 0.1: 0.6373311877250671 error


(None, None, None, None, None, None)

In [23]:
gradient_information(model, train_dataset, iters=50, device=device, subset_size=5000), gradient_information(fgsm_model, train_dataset, iters=50, device=device, subset_size=5000), gradient_information(fgsm_model_small, train_dataset, iters=50, device=device, subset_size=5000), gradient_information(step_ll_model, train_dataset, iters=50, device=device, subset_size=5000)

(tensor(0.0811, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0929, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0448, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0111, device='cuda:0', grad_fn=<MeanBackward0>))

In [24]:
gradient_information(model, test_dataset, iters=50, device=device, subset_size=5000), gradient_information(fgsm_model, test_dataset, iters=50, device=device, subset_size=5000), gradient_information(fgsm_model_small, test_dataset, iters=50, device=device, subset_size=5000), gradient_information(step_ll_model, test_dataset, iters=50, device=device, subset_size=5000)

(tensor(0.0823, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0832, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0253, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0174, device='cuda:0', grad_fn=<MeanBackward0>))